# Data Description


Data url =https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv

*Name : 게임의 이름입니다.*

*Platform : 게임이 지원되는 플랫폼의 이름입니다.*

*Year : 게임이 출시된 연도입니다.*

*Genre : 게임의 장르입니다.*

*Publisher : 게임을 제작한 회사입니다.*

*NA_Sales : 북미지역에서의 출고량입니다.*

*EU_Sales : 유럽지역에서의 출고량입니다.*

*JP_Sales : 일본지역에서의 출고량입니다.*

*Other_Sales : 기타지역에서의 출고량입니다.*

In [ ]:
import pandas as pd
import numpy as np

data_url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"
games = pd.read_csv(data_url, index_col = 0)

games

In [ ]:
games.isnull().sum()

In [ ]:
# 결측치의 비율이 낮기 때문에 제거한다

games = games.dropna()
games

In [ ]:
games = games.reset_index(drop = True)
games

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Year column 의 이상치 여부 시각화

sns.boxplot(data = games['Year'], color = 'red')

plt.show

In [ ]:
# Year column 에서 나타난 이상치를 정상화

def Year(v):
    if v < 30:
        Year = v + 2000
    elif 30 <= v < 100:
        Year = v + 1900
    else:
        Year = v
    return Year


games["Year"] = games["Year"].apply(lambda v: Year(v))

games

In [ ]:
# Sales column 에서 결과값 변경

games['NA_Sales'] = games['NA_Sales'].replace({'K' : '*1e-3', 'M' : '*1'}, regex = True).map(pd.eval)
games['EU_Sales'] = games['EU_Sales'].replace({'K' : '*1e-3', 'M' : '*1'}, regex = True).map(pd.eval)
games['JP_Sales'] = games['JP_Sales'].replace({'K' : '*1e-3', 'M' : '*1'}, regex = True).map(pd.eval)
games['Other_Sales'] = games['Other_Sales'].replace({'K' : '*1e-3', 'M' : '*1'}, regex = True).map(pd.eval)

games

In [ ]:
games1 = games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]*1000

games2 = pd.concat([games.loc[:,['Name', 'Platform', 'Year', 'Genre', 'Publisher']],games1], axis = 1)

# 출고량의 단위를 K(1,000)으로 맞추기
games2 

In [ ]:
games2.isnull().sum()

In [ ]:
games3 = games2.astype({'Year' : int, 'NA_Sales' : int, 'EU_Sales' : int, 'JP_Sales' : int, 'Other_Sales' : int})

games3

In [ ]:
games3.info()

# Sales Columns IQR 활용

In [ ]:
# IQR을 활용 하지 않을 데이터 미리 복사해두기

games4 = games2.copy()

games4['Total_Sales'] = games4['NA_Sales'] + games4['EU_Sales'] + games4['JP_Sales'] + games4['Other_Sales']

games4 = games4.astype({'Year' : int, 'NA_Sales' : int, 'EU_Sales' : int, 'JP_Sales' : int, 'Other_Sales' : int, 'Total_Sales' : int})

games4

In [ ]:
# IQR 분석

NA_plt = games3.plot(kind = 'box', x = 'Year', y = 'NA_Sales')
EU_plt = games3.plot(kind = 'box', x = 'Year', y = 'EU_Sales')
JP_plt = games3.plot(kind = 'box', x = 'Year', y = 'JP_Sales')
Ot_plt = games3.plot(kind = 'box', x = 'Year', y = 'Other_Sales')
print(NA_plt, EU_plt, JP_plt, Ot_plt)

 위 boxplot 들을 보면 이상치가 많고 크게 나타 난다는 것을 알 수 있다.

In [ ]:
games3['NA_Sales'].describe()

In [ ]:
NA1 = games3['NA_Sales'].quantile(0.25)
NA2 = games3['NA_Sales'].quantile(0.75)
NA_iqr = NA2 - NA1
NA_iqr

In [ ]:
# NA_Sales 의 이상치 IQR 최대값으로 변환

def Sales(x):
    if x > NA2 + NA_iqr * 1.5:
        Sales = NA2 + NA_iqr * 1.5
    else:
        Sales = x
    return Sales

games3['NA_Sales'] = games3['NA_Sales'].apply(lambda x: Sales(x))

games3['NA_Sales'].value_counts()

In [ ]:
# 이외 Sales 이상치 변환

EU1 = games3['EU_Sales'].quantile(0.25)
EU2 = games3['EU_Sales'].quantile(0.75)
EU_iqr = EU2 - EU1

JP1 = games3['JP_Sales'].quantile(0.25)
JP2 = games3['JP_Sales'].quantile(0.75)
JP_iqr = JP2 - JP1

Ot1 = games3['Other_Sales'].quantile(0.25)
Ot2 = games3['Other_Sales'].quantile(0.75)
Ot_iqr = Ot2 - Ot1

EU_iqr, JP_iqr, Ot_iqr

In [ ]:
def Sales(x):
    if x > EU2 + EU_iqr * 1.5:
        Sales = EU2 + EU_iqr * 1.5
    else:
        Sales = x
    return Sales

games3['EU_Sales'] = games3['EU_Sales'].apply(lambda x: Sales(x))

games3['EU_Sales'].value_counts()

In [ ]:
def Sales(x):
    if x > JP2 + JP_iqr * 1.5:
        Sales = JP2 + JP_iqr * 1.5
    else:
        Sales = x
    return Sales

games3['JP_Sales'] = games3['JP_Sales'].apply(lambda x: Sales(x))

games3['JP_Sales'].value_counts()

In [ ]:
def Sales(x):
    if x > Ot2 + Ot_iqr * 1.5:
        Sales = Ot2 + Ot_iqr * 1.5
    else:
        Sales = x
    return Sales

games3['Other_Sales'] = games3['Other_Sales'].apply(lambda x: Sales(x))

games3['Other_Sales'].value_counts()

In [ ]:
games3.describe()

In [ ]:
games3

In [ ]:
games3 = games3.astype({'NA_Sales' : int, 'EU_Sales' : int, 'JP_Sales' : int, 'Other_Sales' : int})

games3.info()

In [ ]:
# 앞으로 총 판매량이 필요하다고 생각했기 때문에 총 판매량 column 을 생성한다

games3['Total_Sales'] = games3['NA_Sales'] + games3['EU_Sales'] + games3['JP_Sales'] + games3['Other_Sales']

games3

# 지역에 따라서 선호하는 게임 장르가 다를까

1. groupby 를 사용해서 데이터를 만들고 시각화 해보자

In [ ]:
# 장르별 총 판매량 합산

TO_genre = games3.groupby(by=['Genre'])['Total_Sales'].sum()
TO_genre = TO_genre.reset_index()

# 장르별 북미 판매량 합산

NA_genre = games3.groupby(by = ['Genre'])['NA_Sales'].sum()
NA_genre = NA_genre.reset_index()

# 장르별 유럽 판매량 합산

EU_genre = games3.groupby(by = ['Genre'])['EU_Sales'].sum()
EU_genre = EU_genre.reset_index()

# 장르별 일본 판매량 합산

JP_genre = games3.groupby(by = ['Genre'])['JP_Sales'].sum()
JP_genre = JP_genre.reset_index()

# 장르별 기타지역 판매량 합산

OT_genre = games3.groupby(by = ['Genre'])['Other_Sales'].sum()
OT_genre = OT_genre.reset_index()

# 각 값들을 덮어씌워서 조합한다

TO_genre = TO_genre.combine_first(NA_genre).combine_first(EU_genre).combine_first(JP_genre).combine_first(OT_genre)
TO_genre = TO_genre.set_index('Genre')

In [ ]:
%config InlineBackend.figure_format = 'retina' # 해상도를 높여줌

sns.set(style = 'darkgrid')
plt.figure(figsize = (20, 15))
plt.title('Game Genre Preference by Continent', fontsize = 40)
plt.xlabel('Game Genre', fontsize = 25)
plt.ylabel('Total Sales', fontsize = 25)

sns.lineplot(data = TO_genre, markers = True);

- 일본을 제외한 모든 지역이 1순위, 2순위, 3순위 가 같으며 일본만 다른 형태를 보인다

- 타 지역의 경우에는 1순위 Action, 2순위 Sports, 3순위 Shooter 로 동일하게 나온다

- 일본의 경우에는 Role-Playing 게임이 1순위로 타 지역들의 순위가 한 단계씩 떨어져서 나타난다

2. 테이블셋을 만들고 녹여서 분석하는 방법으로 시각화 해보자

In [ ]:
# 다른 방식의 시각화 방법을 해보자

sales = games3[['Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Total_Sales']]
sales_grouped = sales.groupby(['Genre']).sum()
sales_table = sales_grouped.reset_index()

sales_table

In [ ]:
sales_melted = pd.melt(sales_table, id_vars = ['Genre'], value_vars = sales_table.columns[1:],
                      var_name = 'Country', value_name = 'Sales')
sales_melted.head(10)

In [ ]:
plt.figure(figsize = (20, 15))
sns.barplot(data = sales_melted, x = 'Genre', y = 'Sales', hue = 'Country')  # Bar Plot 의 형태로 장르별 차이를 명확하게 볼 수 있다
plt.title('Game Genre Preference by Continent', loc = 'center', fontsize = 40)
plt.legend(fontsize = 25)
plt.show()

- Barplot 를 사용해서 분석했다

- 위와 같은 결과가 나타난다

In [ ]:
# 지역별 판매량 TOP3만 따로 보자

def Top_genre(games3, s_column):
    Top_games = games3.loc[:, ['Genre', s_column]].sort_values(by = s_column, ascending = False).reset_index(drop = True).head(3) # 내림차순으로 정리한다
    Top_games.columns = ['Genre', 'Sales']
    return Top_games

NA_Top_genre = Top_genre(sales_table, 'NA_Sales')
EU_Top_genre = Top_genre(sales_table, 'EU_Sales')
JP_Top_genre = Top_genre(sales_table, 'JP_Sales')
OT_Top_genre = Top_genre(sales_table, 'Other_Sales')
TO_Top_genre = Top_genre(sales_table, 'Total_Sales')

Top_list = [NA_Top_genre, EU_Top_genre, JP_Top_genre, OT_Top_genre, TO_Top_genre]

columns_list = sales_table.columns[1:]

Top_list, columns_list

In [ ]:
fig, axs = plt.subplots(figsize=(30, 8), nrows=1, ncols=5)

for col, i, data in zip(columns_list, range(len(columns_list)), Top_list):
    axs[i].set_title(col + ' ' + 'TOP3', fontsize = 20)
    sns.barplot(x = 'Genre', y = 'Sales', data = data, ax = axs[i])
    axs[i].set_xlabel('Genre', fontsize = 15)
    axs[i].set_ylabel('Sales', fontsize = 10)

plt.show()

- TOP3 의 장르들만 추려 보았다

- 일본을 제외한 모든 지역들의 선호 순서는 같다

# 연도별 게임의 트렌드가 있을까

In [ ]:
year_genre = games3.groupby(['Year','Genre'])['Total_Sales'].sum().reset_index()

year_genre

In [ ]:
# 연도별 최고 판매량을 기록한 장르를 찾아보자

year_sales = year_genre.groupby(['Year'])['Total_Sales'].transform(max) == year_genre['Total_Sales']
year_sales_max = year_genre[year_sales].reset_index(drop=True)

year_sales_max.head(10)

In [ ]:
max_genre = year_sales_max['Genre'].values # 연도별 최고 장르만 뽑아둔다

max_genre

In [ ]:
plt.figure(figsize = (30, 12))
p = sns.barplot(x = 'Year', y = 'Total_Sales', data = year_sales_max)
index = 0

for value in year_sales_max['Total_Sales']:
  p.text(index, value + 1, str(max_genre[index] + ' -- ' + str(value)), color = 'grey', size = 12, rotation = 65, ha = 'center') # Text 를 정방향으로 하면 겹치기 때문에 안겹치는 선인 65도 돌렸고, horizontal align 을 center 로 해서 중심축을 가운데에 둔다
  index += 1

plt.title('Best Game Genre of the Year', fontsize = 30)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Total Sales Value', fontsize = 15)

plt.show()

- 연도별 게임의 트렌드는 존재한다

- 2000년대에 들어서 Action 장르가 강세를 나타냈다

- 또한 1996년을 기점으로 게임산업이 성장했지만 13년을 기점으로 하향세를 보인다

# 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
best_games = games3.sort_values(['Total_Sales'], ascending = False) # Total Sales 를 내림차순으로 정렬한다

best_games20 = best_games.head(20)

best_games20 = best_games20[['Name', 'Platform', 'Genre', 'Total_Sales']].reset_index(drop = True)

best_games20

- Total Sales 를 보면 동일하다

- 이상치를 균일화 했기 때문이라고 판단되기 때문에 이상치를 건드리지 않는 데이터를 사용한다

- 미리 만들어둔 games4 데이터를 활용한다

In [ ]:
best_games1 = games4.sort_values(['Total_Sales'], ascending = False)

best20_games = best_games1.head(20)

best20_games = best20_games[['Name', 'Platform', 'Genre', 'Publisher', 'Total_Sales']].reset_index(drop = True)

Ranking = ['No.' + str(x) for x in range(1, 21)]

best20_games['Ranking'] = Ranking

best20_games

- games4 데이터를 사용한 경우 Wii Sports 가 가장 높은 판매량을 나타내고 있었다

- 출고량이 높은 게임에 대해서는 앞으로 games4 데이터를 기반으로 한 데이터를 사용하도록 한다

In [ ]:
plt.figure(figsize = (30,10))
R = sns.barplot(x = 'Ranking', y = 'Total_Sales', data = best20_games)
i = 0
for Name, Platform, Genre, Publisher, Sales in zip(best20_games.Name, best20_games.Platform, best20_games.Genre, best20_games.Publisher, best20_games.Total_Sales):
    R.text(x = i, y = Sales + 10, s = (Name), color = 'black', rotation = 45, ha = 'center', fontsize = 15)                       # 타이틀명이 겹쳐서 보이기 때문에 각도를 틀었다
    R.text(x = i, y = Sales - 10000, s = ('\n' + Platform + '\n' + Genre + '\n' + Publisher), color = 'black', ha = 'center', fontsize = 12)
    i+=1

R.set_title('Sales Top 20 Games', fontsize = 30)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.xlabel('Game Title', fontsize = 15)
plt.ylabel('Sales Data', fontsize = 15)
plt.show()

- Wii Sports 라는 게임이 압도적인 매출을 기록하고있다

- Wii 플랫폼의 Sports 장르라는 것을 알아 볼 수 있다

# 일정 기간 동안의 게임 순위 알아보기

- 연간 Total Sales Data 를 시각화 해서 판매량이 좋던 시기를 알아보자

In [ ]:
games_sales_data = games4.groupby('Year').sum()

sns.set(style = 'darkgrid')
plt.figure(figsize = (20, 10))
plt.title('Total Sales Data of the Year', fontsize = 30)
plt.xlabel('Year', fontsize = 20)
plt.ylabel('Total Sales', fontsize = 20)

sns.lineplot(data = games_sales_data, markers = True);

- 위에서 볼 수 있듯, 2002 ~ 2012 년 사이의 데이터가 가장 고평가 되는 구간인 것을 알 수 있다

- 북미 시장이 가장 큰 시장이라는 것을 알 수 있다

- 2008년의 Total Sales 가 가장 많은데, 북미시장의 점유율은 50%를 넘겼다는 것을 알 수 있다

- 2002년 부터 2012년 까지의 데이터를 이용해 게임 판매량 순위를 알아보자

In [ ]:
# 2012년까지의 순위를 먼저 구한다

year_sales_top10 = games4[games4.Year <= 2012].sort_values('Total_Sales', ascending = False)

# 위의 값에서 2002년 부터로 재 조정한다

year_sales_top10 = year_sales_top10[year_sales_top10.Year >= 2002].sort_values('Total_Sales', ascending = False).head(10)

year_sales_top10 = year_sales_top10[['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Total_Sales']].reset_index(drop = True)

# Ranking 값이 20위였으니 10위 단위로 새로 짜준다

Ranking_10 = ['No.' + str(x) for x in range(1, 11)]
year_sales_top10['Ranking'] = Ranking_10

year_sales_top10

In [ ]:
plt.figure(figsize = (30,10))
R1 = sns.barplot(x = 'Ranking', y = 'Total_Sales', data = year_sales_top10)
i = 0
for Name, Platform, Year, Genre, Publisher, Sales in zip(year_sales_top10.Name, year_sales_top10.Platform, year_sales_top10.Year, year_sales_top10.Genre, year_sales_top10.Publisher, year_sales_top10.Total_Sales):
    R1.text(x = i, y = Sales + 10, s = (Name), color = 'black', ha = 'center', fontsize = 15)
    R1.text(x = i, y = Sales - 10000, s = (Platform + '\n' + str(Year) + 'year' + '\n' + Genre + '\n' + Publisher), color = 'black', ha = 'center', fontsize = 12)
    i+=1

R1.set_title('Top10 Games during 2002 - 2012 Season', fontsize = 30)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.xlabel('Game Title', fontsize = 15)
plt.ylabel('Sales Data', fontsize = 15)
plt.show()

- 위 분석으로 보면 전체 기간에서 나왔던 Super Mario Bros 는 옛날 게임이란 것을 알 수 있다

- 전체 자료와 위 기간 자료를 본 결과 닌텐도 게임은 항상 잘 된다고 볼 수 있다

- 최근 10년 간의 Top 5 게임만을 찾아보자

- 위 데이터의 마지막 구간이 2020 년으로 나오지만 2022년 현재를 기준으로 10년 기간을 잡는다

In [ ]:
last10_year_top5 = games4[games4.Year >= 2013].sort_values('Total_Sales', ascending = False).head(5)

last10_year_top5 = last10_year_top5[['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Total_Sales']].reset_index(drop = True)

Ranking_5 = ['No.' + str(x) for x in range(1, 6)]

last10_year_top5['Ranking'] = Ranking_5

last10_year_top5

In [ ]:
plt.figure(figsize = (20,10))
R2 = sns.barplot(x = 'Ranking', y = 'Total_Sales', data = last10_year_top5)
i = 0
for Name, Platform, Year, Genre, Publisher, Sales in zip(last10_year_top5.Name, last10_year_top5.Platform, last10_year_top5.Year, last10_year_top5.Genre, last10_year_top5.Publisher, last10_year_top5.Total_Sales):
    R2.text(x = i, y = Sales + 10, s = (Name), color = 'black', ha = 'center', fontsize = 15)
    R2.text(x = i, y = Sales - 10000, s = (Platform + '\n' + str(Year) + 'year' + '\n' + Genre + '\n' + Publisher), color = 'black', ha = 'center', fontsize = 12)
    i+=1

R2.set_title('Last 10 Years TOP 5 Games', fontsize = 30)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.xlabel('Game Title', fontsize = 15)
plt.ylabel('Sales Data', fontsize = 15)
plt.show()

- 최근 10년 간의 게임은 PS 시리즈의 플랫폼이 강세를 나타낸다

- Grand Theft Auto V 가 다른 플랫폼으로  1, 2, 4순위에 오른 것으로 보아 압도적인 판매량이 예상된다

- Nintendo 의 게임은 포켓몬 시리즈가 유일하다

# 플랫폼에 대한 분석을 해보자

- 전체 기간 동안에 대한 지역별 플랫폼 판매량을 시각화한다

- 어떤 플랫폼에서의 판매량이 좋은지를 알아본다

In [ ]:
# 지역별 플랫폼 판매량 분석 - Total 판매량과 함께 비교

platform_sales = games4[['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Total_Sales']]
platform_sales1 = platform_sales.groupby('Platform').sum()
platform_sales2 = platform_sales1.reset_index()
platform_sales3 = pd.melt(platform_sales2, id_vars = ['Platform'], value_vars = platform_sales.columns[1:], var_name = 'Continent', value_name = 'Sales')

plt.figure(figsize=(25,10))
pt = sns.barplot(x = 'Platform', y = 'Sales', hue = 'Continent', data = platform_sales3)
pt.set_title('Sales Data by Platform', fontsize = 25)
pt.legend(fontsize = 20)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.xlabel('Platform', fontsize = 20)
plt.ylabel('Sales', fontsize = 20)
plt.show()

- PS2가 가장 강세를 보이고 있으며 PS 시리즈가 좋은 모습을 보인다

- 그 외에는 X360과 Wii, DS 등이 좋은 모습을 보인다

In [ ]:
# 최근 10년간의 지역별 플랫폼 판매량을 분석한다

last10_years = games4[games4.Year >= 2013]

last_platform = last10_years[['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Total_Sales']]
last_platform1 = last_platform.groupby('Platform').sum()
last_platform2 = last_platform1.reset_index()
last_platform3 = pd.melt(last_platform2, id_vars = ['Platform'], value_vars = platform_sales.columns[1:], var_name = 'Continent', value_name = 'Sales')

plt.figure(figsize=(15,7))
pt1 = sns.barplot(x = 'Platform', y = 'Sales', hue = 'Continent', data = last_platform3)
pt1.set_title('Sales Data by Platform', fontsize = 25)
pt1.legend(fontsize = 20)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.xlabel('Platform', fontsize = 20)
plt.ylabel('Sales', fontsize = 20)
plt.show()

- 최근 10년간의 데이터를 보면 많은 플랫폼들이 사라져있다

- 하지만 여기서도 PS 시리즈가 가장 강세를 보이고 있으며 X시리즈와 3DS가 좋은 모습을 보인다

- 지난 자료와 비교해 보았을 때, PC게임의 시장 파이는 크게 줄어든 모습을 보인다

In [ ]:
def top3_platform(games, sales):
  last10_top3 = games.loc[:, ['Platform', sales]].sort_values(by = sales, ascending = False).reset_index(drop = True).head(3)
  last10_top3.columns = ['Platform', 'Sales']
  return last10_top3

NA_10y3t = top3_platform(last_platform2, 'NA_Sales')
EU_10y3t = top3_platform(last_platform2, 'EU_Sales')
JP_10y3t = top3_platform(last_platform2, 'JP_Sales')
OT_10y3t = top3_platform(last_platform2, 'Other_Sales')
TO_10y3t = top3_platform(last_platform2, 'Total_Sales')

platform_list = [NA_10y3t, EU_10y3t, JP_10y3t, OT_10y3t, TO_10y3t]
pt_columns_list = last_platform.columns[1:]

fig, axs = plt.subplots(figsize = (30, 7), nrows = 1, ncols = 5)
for col, i, data in zip(pt_columns_list, range(len(pt_columns_list)), platform_list):
    axs[i].set_title(col + ' ' + 'TOP 3', fontsize = 20)
    sns.barplot(x = 'Platform', y = 'Sales', data = data, ax = axs[i])
    axs[i].tick_params(labelsize = 12)
    axs[i].set_xlabel('Platform', fontsize = 12)
    axs[i].set_ylabel('Sales', fontsize = 12)
    
plt.show()

- 최근 10년 간의 지역별 TOP 3 플랫폼을 찾아보았다

- 일본을 제외하고 PS4가 가장 판매량이 높은 플랫폼으로 나타났다

- 전체적으로는 PS 시리즈가 가장 좋은 모습을 나타내고 그 다음은 X 시리즈 플랫폼으로 보인다

# 다음 분기에 어떤 게임을 설계해야 할까

- 지금까지 과거 게임 데이터를 분석해 보았다

- PC 플랫폼의 판매량은 최근 굉장히 낮아지고 있기 때문에 앞으로는 PC보다는 콘솔 플랫폼 이용이 가능한 게임을 위주로 설계하는게 좋아보인다

- 데이터들을 본 결과 PS 시리즈 플랫폼과 X 시리즈 플랫폼을 이용 가능한 게임이 현재 가장 가능성이 있다고 판단된다

- 지역별 선호도는 일본을 제외한 경우 다 같은 모습을 보이며 판매량 점유율에 있어서 북미, 유럽, 일본 순으로 나타나고 북미의 점유율이 40% 이상을 보이기 때문에 북미, 유럽의 게이머들에 맞는 장르를 선택하는 것이 좋아 보인다

- 게임의 장르는 Action 장르가 최근 강세를 보이기에 Action 장르의 게임을 만드는 것이 가장 미래가 밝을 것이라고 판단된다

- 결과적으로 우리는 PS시리즈, X시리즈의 플랫폼 이용을 우선적으로 하는 Action 장르의 게임을 설계하고 북미, 유럽 지역 문화에 맞는 게임을 방향으로 잡는 것이 가장 바람직하다고 판단된다.